# 8 Deep Learning

8.1 Use a pre-trained ResNet50 and VGG16 on Keras to implement an inference of the test image in this directory!

* Example: <https://keras.io/applications/>


8.2 Use a pre-trained ResNet50 and VGG16 on PyTorch to implement an inference of the test image in this directory!

* Example: http://pytorch.org/docs/master/torchvision/models.html

8.3 Compare the inference times of both networks and frameworks!

8.4 Find two positive and negative sample images that are correctly/incorrectly classified!


In [28]:
import os, sys
import tensorflow as tf
os.chdir(os.path.join(os.environ["HOME"], "exercise-students-2020/08_DeepLearning"))
os.environ["KERAS_BACKEND"]="tensorflow"
import keras
from keras.applications.resnet50 import ResNet50
from keras.applications.resnet50 import preprocess_input as resnet_preprocess_input, decode_predictions as resnet_decode_predictions
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input as vgg_preprocess_input, decode_predictions as vgg_decode_predictions
import numpy as np
from keras.preprocessing.image import load_img, img_to_array

In [20]:
resnet = ResNet50()
vgg = VGG16()

553467904/553467096 [==============================] - 49s 0us/step


In [21]:
image = img_to_array(load_img('test_image.jpg' , target_size=(224, 224)))
image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))

In [22]:
resnet_decode_predictions(resnet.predict(resnet_preprocess_input(image)))

[[('n04604644', 'worm_fence', 0.07046478),
  ('n02793495', 'barn', 0.0686209),
  ('n03000134', 'chainlink_fence', 0.061941594),
  ('n04326547', 'stone_wall', 0.05747776),
  ('n03930313', 'picket_fence', 0.05441839)]]

In [29]:
vgg_decode_predictions(vgg.predict(vgg_preprocess_input(image)))

[[('n02793495', 'barn', 0.3092531),
  ('n03028079', 'church', 0.23857628),
  ('n03743016', 'megalith', 0.09048893),
  ('n04604644', 'worm_fence', 0.06188944),
  ('n03891251', 'park_bench', 0.05850323)]]